# 基本思路：
    1、获取stockbasic和stockearningreport的全量数据
    2、stockbasic根据所在市场进行过滤，只要主板和中小板的
    3、stockearingreport进行筛选，条件为：
        3.1 基于2中的主板和中小板数据
        3.2 只要最近3年的数据
        3.3 仅要年报（1231）这一天
    4、完成后，stockearningreport根据code进行聚类，并获取net_proft_yoy_raise的最小，均值和最大值
    5、根据group后的net_proft_yoy_raise最小值进行筛选，暂定条件为大于30%

In [43]:
import os
import sys

mod_path = os.path.abspath('./../dataUpdate/')
print(mod_path)
sys.path.append('/Users/roland/vscode_program/backtrader_learning')
sys.path.insert(0,'/Users/roland/vscode_program/backtrader_learning/dataUpdate')

/Users/roland/vscode_program/backtrader_learning/dataUpdate


In [44]:
from sqlalchemy.orm import Session
from dataUpdate import engine
from dataUpdate.model.basicinfo import StockEarningReport,StockBasic
import pandas as pd
import datetime

In [45]:
sess = Session(engine)

In [46]:
# 1、获取所有报表数据
sql = sess.query(StockEarningReport)
res_df = pd.read_sql(sql.statement,sql.session.bind)
res_df.head(3)

,id,code,earning_per_share,revenue,revenue_yoy_rise,revenue_sos_rise,net_profit,net_profit_yoy_rise,net_profit_sos_rise,return_on_equity,operating_cash_flow_per_share,gross_profit_ratio,year,date
0,1,000617,-0.04,3.046625e+08,10.749529,-47.6992,-10572685.25,-134.422353,-231.3498,-1.27,-0.100000,3.111316,2010,0331
1,2,000757,-0.08,1.319204e+08,101.189055,16.3653,-25948235.48,16.468424,-146.0735,0.00,-0.042300,12.714494,2010,0331
2,3,301041,NaN,7.030412e+07,NaN,NaN,4112270.94,NaN,NaN,NaN,-0.169109,15.906955,2010,0331


In [47]:
# 2、获取主板+中小板数据
sql2 = sess.query(StockBasic.code,StockBasic.symbol,StockBasic.name,StockBasic.circulating_market_cap,StockBasic.sw_l1).filter(StockBasic.market.in_(['主板','中小板']))
res_df2 = pd.read_sql(sql2.statement,sql2.session.bind)
res_df2.head(3)

,code,symbol,name,circulating_market_cap,sw_l1
0,sh.600000,600000,浦发银行,3137.740,银行I
1,sh.600004,600004,白云机场,253.492,交通运输I
2,sh.600006,600006,东风汽车,128.400,汽车I


In [48]:
# 3、仅获取主板和中小板的，3年年份的年报数据
annul_report = res_df[(res_df['year'].isin([2016,2017,2018,2019,2020])) & (res_df['date']=='1231') & (res_df['code'].isin(res_df2['symbol']))]
annul_report.shape

(15520, 14)

In [49]:
annul_report[annul_report['code']=='601919']

,id,code,earning_per_share,revenue,revenue_yoy_rise,revenue_sos_rise,net_profit,net_profit_yoy_rise,net_profit_sos_rise,return_on_equity,operating_cash_flow_per_share,gross_profit_ratio,year,date
127466,127467,601919,-0.97,7.116018e+10,9.417290,12.4248,-9.906004e+09,-2210.796045,65.8929,-44.19,0.148736,-0.991593,2016,1231
159178,159179,601919,0.26,9.046396e+10,27.127217,-5.2420,2.661936e+09,126.871945,-108.4755,13.77,0.694190,8.687793,2017,1231
192670,192671,601919,0.12,1.208295e+11,33.566485,4.4383,1.230026e+09,-53.792034,-55.2878,5.52,0.795865,8.462042,2018,1231
225298,225299,601919,0.56,1.510567e+11,25.016363,-0.1942,6.764105e+09,449.920000,428.7379,21.57,1.729460,10.742339,2019,1231
258685,258686,601919,0.81,1.712588e+11,13.373888,22.6643,9.927098e+09,46.760000,122.8038,25.05,3.673106,14.208792,2020,1231


In [50]:
# 4、根据code进行groupby，并计算年净利润增长率的最小，均值和最大值
annul_report_groupby = annul_report.groupby('code').agg({'net_profit_yoy_rise':['min','mean','max']})

In [51]:
# 5、strategy1: 最低净利润增长也要大于30
res1 = annul_report_groupby[annul_report_groupby['net_profit_yoy_rise']['min']>20]

In [52]:
res1.shape

(54, 3)

In [53]:
res1.head()

net_profit_yoy_rise                        
                       min        mean         max
code                                              
000528           28.740000  174.800098  555.560864
000568           29.380000   32.081093   36.267338
000661           26.106196   52.536470   76.360000
000799           22.597805   41.975163   64.150000
001203           38.745537   66.109749  102.961932

In [15]:
# 6、将结果进行inner join
res2 = pd.merge(left=res1,right=res_df2,left_on='code',right_on='symbol',how='inner')

/Users/roland/anaconda3/envs/py374/lib/python3.7/site-packages/pandas/core/reshape/merge.py:648: UserWarning: merging between different levels can give an unintended result (2 levels on the left,1 on the right)
  warnings.warn(msg, UserWarning)


In [16]:
# 7、排序并展示
res2.sort_values(by='circulating_market_cap',inplace=True,ascending=False)
res2.head()

,"(net_profit_yoy_rise, min)","(net_profit_yoy_rise, mean)","(net_profit_yoy_rise, max)",code,symbol,name,circulating_market_cap,sw_l1
144,22.285194,34.811731,49.580000,sh.601888,601888,中国国旅,7572.87,休闲服务I
8,29.380000,32.939113,36.267338,sz.000568,000568,泸州老窖,4597.78,食品饮料I
111,36.250000,103.936743,192.330230,sh.600031,600031,三一重工,4066.41,机械设备I
132,28.630000,46.341950,56.390000,sh.600809,600809,山西汾酒,3840.38,食品饮料I
67,53.280000,62.486099,73.130000,sz.002475,002475,立讯精密,3701.54,电子I


In [17]:
res2['sw_l1'].value_counts()

机械设备I    25
化工I      24
医药生物I    18
电子I      16
公用事业I    13
计算机I     12
食品饮料I    11
电气设备I    10
房地产I      9
轻工制造I     9
建筑装饰I     6
建筑材料I     6
有色金属I     5
传媒I       5
家用电器I     4
交通运输I     4
采掘I       3
汽车I       3
通信I       2
纺织服装I     2
非银金融I     2
综合I       2
商业贸易I     2
国防军工I     2
休闲服务I     1
钢铁I       1
Name: sw_l1, dtype: int64

In [21]:
res2[res2.sw_l1=='机械设备I']

NameError: name 'res2' is not defined

In [5]:
# 获取股票的年最大最小值
from dataUpdate.model.tradingInfo import TradingInfo

sess = Session(engine)


In [15]:
c_left = datetime.datetime(2019,1,1)
c_right = datetime.datetime(2019,12,31)

sql = sess.query(TradingInfo).filter(TradingInfo.code=='sh.600031',TradingInfo.time.between(c_left,c_right))
res_frame = pd.read_sql(sql.statement,sql.session.bind)
res_frame.head()

,id,code,time,open,close,high,low,volume,amount
0,710433,sh.600031,2019-01-02 10:00:00,8.31,8.16,8.31,8.16,6870680.0,56401800.0
1,710434,sh.600031,2019-01-02 10:30:00,8.17,8.07,8.18,8.04,7920900.0,64226300.0
2,710435,sh.600031,2019-01-02 11:00:00,8.07,8.04,8.08,8.01,8078170.0,64927000.0
3,710436,sh.600031,2019-01-02 11:30:00,8.04,8.10,8.11,8.03,4302700.0,34699300.0
4,710437,sh.600031,2019-01-02 13:30:00,8.09,8.02,8.11,8.00,6967100.0,55999700.0


In [19]:
year_open = res_frame['open'][0]
max_raise = res_frame['high'].max()/year_open-1
max_down = 1-res_frame['low'].min()/year_open

print(max_raise,max_down)

1.030084235860409 0.0649819494584839


In [20]:
res_frame['high'].max(),res_frame['low'].min()

(16.87, 7.77)

In [80]:
def getYearOCHL(sess,code,start_year,end_year):
    """获取指定股票年化的OHCL数据

    Args:
        sess (session): 数据库连接
        code (string): 股票代码，例如 sh.600000
        start_year (int): 起始年份，例如 2015
        end_year (int): 结束年份，2020；即会获得【2015，2020】的年化olch数据

    Returns:
        dataframe : 返回数据结果
    """
    c_left = datetime.datetime(start_year,1,1)
    c_right = datetime.datetime(end_year,12,13)
    sql = sess.query(TradingInfo).filter(TradingInfo.code==code,TradingInfo.time.between(c_left,c_right))
    res = pd.read_sql(sql.statement,sql.session.bind)
    res['year'] = res['time'].apply(lambda x :x.year)

    t_open = res.groupby('year').first()
    t_close = res.groupby('year').last()
    temp = pd.merge(left=t_open,right=t_close,left_on='year',right_on='year',how='inner',suffixes=("_open","_close"))
    temp = temp[['code_open','open_open','close_close']]
    temp.rename(mapper={'code_open':'code','open_open':'open','close_close':'close'},inplace=True)
    return temp

In [66]:
def getEarningReport(sess,start_year,end_year):
    """拿到年报

    Args:
        sess (session): 数据库连接
        start_year (int): 起始年份
        end_year (int): 终止年份

    Returns:
        dataframe: 年报dataframe
    """

    # 1、获取所有报表数据
    sql = sess.query(StockEarningReport)
    res_df = pd.read_sql(sql.statement,sql.session.bind)

    # 2、获取主板+中小板数据
    sql2 = sess.query(StockBasic.code,StockBasic.symbol,StockBasic.name,StockBasic.circulating_market_cap,StockBasic.sw_l1).filter(StockBasic.market.in_(['主板','中小板']))
    res_df2 = pd.read_sql(sql2.statement,sql2.session.bind)

    # 3、仅获取主板和中小板的，3年年份的年报数据
    year_range = list(range(start_year,end_year+1))
    annul_report = res_df[(res_df['year'].isin(year_range)) & (res_df['date']=='1231') & (res_df['code'].isin(res_df2['symbol']))]

    # 4、根据code进行groupby，并计算年净利润增长率的最小，均值和最大值
    # annul_report_groupby = annul_report.groupby('code').agg({'net_profit_yoy_rise':['min','mean','max']})
    return annul_report

In [73]:
start_year,end_year = 2015,2020
annual_report = getEarningReport(sess,start_year,end_year)
annul_report.head()

,id,code,earning_per_share,revenue,revenue_yoy_rise,revenue_sos_rise,net_profit,net_profit_yoy_rise,net_profit_sos_rise,return_on_equity,operating_cash_flow_per_share,gross_profit_ratio,year,date
117605,117606,000839,0.0587,3.927050e+09,39.451942,40.8674,2.302017e+08,-34.540125,-75.8744,3.48,-0.060729,19.830438,2016,1231
117606,117607,002412,0.2870,7.984669e+08,2.219748,17.9409,8.493916e+07,-13.395793,137.3835,6.74,0.581136,71.155351,2016,1231
117607,117608,002396,0.5926,5.687658e+09,25.930517,60.9749,3.194580e+08,21.772219,4.1714,11.62,1.008150,43.331039,2016,1231
117608,117609,002247,0.3500,1.438586e+09,60.703398,-43.8099,2.504799e+08,191.939636,-104.8860,12.48,0.359478,27.258591,2016,1231
117609,117610,002052,-0.7600,5.634953e+08,-42.869130,14.0770,-5.686676e+08,-1234.058555,-398.3615,-49.47,0.311252,15.267732,2016,1231


In [74]:
annual_report['code_year'] = annual_report.apply(lambda row:row['code']+str(row['year']),axis=1)
annual_report.head()

,id,code,earning_per_share,revenue,revenue_yoy_rise,revenue_sos_rise,net_profit,net_profit_yoy_rise,net_profit_sos_rise,return_on_equity,operating_cash_flow_per_share,gross_profit_ratio,year,date,code_year
89039,89040,000839,0.0897,2.816060e+09,-1.138510,124.2233,3.516684e+08,126.007348,32.5832,5.26,-0.009105,15.624139,2015,1231,0008392015
89040,89041,002247,0.1700,8.951808e+08,3.244948,-3.9860,8.579853e+07,-13.377243,-34.3514,8.40,0.507729,23.921510,2015,1231,0022472015
89041,89042,002052,-0.0600,9.863238e+08,-38.425417,23.2399,-4.262688e+07,89.082840,-8839.3648,7.75,0.265373,29.314460,2015,1231,0020522015
89042,89043,600965,0.2000,1.343812e+09,9.810590,-25.8824,1.663804e+08,26.190000,-67.2116,10.58,0.315262,40.912970,2015,1231,6009652015
89043,89044,002642,0.3503,1.546195e+09,3.235722,77.4516,2.089858e+08,26.141574,886.4718,7.95,0.072094,39.229656,2015,1231,0026422015


In [75]:
annual_report[annual_report['code']=='600031']

,id,code,earning_per_share,revenue,revenue_yoy_rise,revenue_sos_rise,net_profit,net_profit_yoy_rise,net_profit_sos_rise,return_on_equity,operating_cash_flow_per_share,gross_profit_ratio,year,date,code_year
91360,91361,600031,0.0007,2.347034e+10,-23.045995,1.8929,4.961000e+06,-80.458992,133.7835,0.02,0.354088,24.622503,2015,1231,6000312015
125319,125320,600031,0.0267,2.328007e+10,-0.810687,28.8608,2.034570e+08,4001.128805,-21.1788,0.89,0.424324,26.205546,2016,1231,6000312016
159143,159144,600031,0.2733,3.833509e+10,64.669108,13.6826,2.092253e+09,928.351445,-54.9368,8.69,1.116884,30.074887,2017,1231,6000312017
189425,189426,600031,0.7907,5.582150e+10,45.614653,13.8213,6.116288e+09,192.330230,-17.5233,21.45,1.349479,30.621794,2018,1231,6000312018
225391,225392,600031,1.3740,7.623284e+10,35.549483,10.9137,1.132592e+10,83.230000,-15.1035,27.16,1.410321,32.687948,2019,1231,6000312019
258778,258779,600031,1.8397,1.000543e+11,31.248261,9.8765,1.543146e+10,36.250000,-25.1463,29.64,1.576092,29.818251,2020,1231,6000312020


In [96]:
annual_report.shape

(18503, 15)

In [82]:
from dataUpdate.util import ak_util
from tqdm import tqdm
temp_df = None
for code in tqdm(set(list(annual_report['code']))):
    code = ak_util.convertAkCode2BSCode(code)
    yearOCHLdf = getYearOCHL(sess,code,start_year,end_year)
    if temp_df is None:
        temp_df = yearOCHLdf
    else:
        temp_df = pd.concat([temp_df,yearOCHLdf])

temp_df.shape

100%|██████████| 3109/3109 [52:43<00:00,  1.02s/it]


(16030, 3)

In [88]:
temp_df.head()

,code_open,open_open,close_close
year,,,
2015,sh.600159,4.66,6.17
2016,sh.600159,6.29,5.34
2017,sh.600159,5.32,3.82
2018,sh.600159,3.83,2.41
2019,sh.600159,2.42,2.55


In [103]:
temp_df['year'] = temp_df.index
temp_df['code_year'] = temp_df.apply(lambda row:row['code_open'].split(".")[1]+str(row['year']),axis=1)
temp_df['annul_rise_percent'] = temp_df.apply(lambda row: (row['close_close']-row['open_open'])/row['open_open'],axis=1)
temp_df.head()

,code_open,open_open,close_close,code_year,year,annul_rise_percent
year,,,,,,
2015,sh.600159,4.66,6.17,6001592015,2015,0.324034
2016,sh.600159,6.29,5.34,6001592016,2016,-0.151033
2017,sh.600159,5.32,3.82,6001592017,2017,-0.281955
2018,sh.600159,3.83,2.41,6001592018,2018,-0.370757
2019,sh.600159,2.42,2.55,6001592019,2019,0.053719


In [104]:
inner_res = pd.merge(left=annual_report,right=temp_df,left_on='code_year',right_on='code_year',how='inner')
print(inner_res.shape)
print(inner_res.head())

(16030, 20)
      id    code  earning_per_share       revenue  revenue_yoy_rise  \
0  89040  000839             0.0897  2.816060e+09         -1.138510   
1  89041  002247             0.1700  8.951808e+08          3.244948   
2  89042  002052            -0.0600  9.863238e+08        -38.425417   
3  89043  600965             0.2000  1.343812e+09          9.810590   
4  89044  002642             0.3503  1.546195e+09          3.235722   

   revenue_sos_rise    net_profit  net_profit_yoy_rise  net_profit_sos_rise  \
0          124.2233  3.516684e+08           126.007348              32.5832   
1           -3.9860  8.579853e+07           -13.377243             -34.3514   
2           23.2399 -4.262688e+07            89.082840           -8839.3648   
3          -25.8824  1.663804e+08            26.190000             -67.2116   
4           77.4516  2.089858e+08            26.141574             886.4718   

   return_on_equity  operating_cash_flow_per_share  gross_profit_ratio  \
0           

In [105]:
inner2 = inner_res[['code','code_open','net_profit_yoy_rise','open_open','close_close','annul_rise_percent','year_x']]
inner2.head()

,code,code_open,net_profit_yoy_rise,open_open,close_close,annul_rise_percent,year_x
0,000839,sz.000839,126.007348,11.27,20.35,0.805679,2015
1,002247,sz.002247,-13.377243,10.49,20.71,0.974261,2015
2,002052,sz.002052,89.082840,9.12,14.65,0.606360,2015
3,600965,sh.600965,26.190000,8.94,16.30,0.823266,2015
4,002642,sz.002642,26.141574,28.22,57.89,1.051382,2015


In [128]:
inner2[inner2.code=='600031']

,code,code_open,net_profit_yoy_rise,open_open,close_close,annul_rise_percent,year_x
255,600031,sh.600031,-80.458992,9.95,6.58,-0.338693,2015
3531,600031,sh.600031,4001.128805,6.45,6.10,-0.054264,2016
6524,600031,sh.600031,928.351445,6.10,9.07,0.486885,2017
8896,600031,sh.600031,192.330230,9.16,8.34,-0.089520,2018
12366,600031,sh.600031,83.230000,8.31,17.05,1.051745,2019
15334,600031,sh.600031,36.250000,17.29,31.50,0.821862,2020


In [135]:
import math
p_cnt =0
t_cnt = 0
wave = 0
for code in set(inner2.code):
    temp3 = inner2[inner2.code==code]
    res4 = temp3['net_profit_yoy_rise'].rolling(window=3,min_periods=3).min()
    for idx in range(len(res4)-1):
        value = res4.values[idx]
        if not math.isnan(value):
            if value>20:
                t_cnt+=1
                wave+=temp3.iloc[idx+1]['annul_rise_percent']
                # if temp3.iloc[idx+1]['annul_rise_percent']>0:
                #     p_cnt+=1
            # print(idx,value,temp3.iloc[idx]['annul_rise_percent'])
wave/t_cnt

0.003471987021152901

In [126]:
p_cnt,t_cnt

(3827, 10161)

In [119]:
res4 = temp3['net_profit_yoy_rise'].rolling(window=3,min_periods=3).min()
res4

255             NaN
3531            NaN
6524     -80.458992
8896     192.330230
12366     83.230000
15334     36.250000
Name: net_profit_yoy_rise, dtype: float64

In [122]:
# res4.dropna(inplace=True)
import math

for idx,value in enumerate(res4):
    # print(type(value),math.isnan(value))
    if not math.isnan(value):
        print(idx,value,temp3.iloc[idx]['annul_rise_percent'])

<class 'float'> True
0 nan -0.3386934673366834
<class 'float'> True
1 nan -0.05426356589147295
<class 'float'> False
2 -80.4589921687 0.48688524590163945
<class 'float'> False
3 192.3302296615 -0.08951965065502186
<class 'float'> False
4 83.23 1.0517448856799037
<class 'float'> False
5 36.25 0.8218623481781377


In [65]:
c_left = datetime.datetime(2016,1,1)
c_right = datetime.datetime(2020,12,13)
sql = sess.query(TradingInfo).filter(TradingInfo.code=='sh.600031',TradingInfo.time.between(c_left,c_right))
res = pd.read_sql(sql.statement,sql.session.bind)
res.head()


,id,code,time,open,close,high,low,volume,amount
0,704585,sh.600031,2016-01-04 10:00:00,6.45,6.47,6.49,6.43,16214700.0,104739000.0
1,704586,sh.600031,2016-01-04 10:30:00,6.48,6.37,6.48,6.31,13477200.0,86100500.0
2,704587,sh.600031,2016-01-04 11:00:00,6.37,6.34,6.37,6.30,8238900.0,52159300.0
3,704588,sh.600031,2016-01-04 11:30:00,6.34,6.27,6.34,6.21,8824200.0,55390500.0
4,704589,sh.600031,2016-01-04 13:30:00,6.27,6.15,6.28,6.14,9163900.0,56799100.0


In [55]:
res['year'] = res['time'].apply(lambda x :x.year)
res.head()

,id,code,time,open,close,high,low,volume,amount,year
0,704585,sh.600031,2016-01-04 10:00:00,6.45,6.47,6.49,6.43,16214700.0,104739000.0,2016
1,704586,sh.600031,2016-01-04 10:30:00,6.48,6.37,6.48,6.31,13477200.0,86100500.0,2016
2,704587,sh.600031,2016-01-04 11:00:00,6.37,6.34,6.37,6.30,8238900.0,52159300.0,2016
3,704588,sh.600031,2016-01-04 11:30:00,6.34,6.27,6.34,6.21,8824200.0,55390500.0,2016
4,704589,sh.600031,2016-01-04 13:30:00,6.27,6.15,6.28,6.14,9163900.0,56799100.0,2016


In [56]:
a = res.groupby('year').first()

In [57]:
b=res.groupby('year').last()

In [58]:
c = pd.merge(left=a,right=b,left_on='year',right_on='year',how='inner',suffixes=("_open","_close"))
c = c[['code_open','open_open','close_close']]
c

,code_open,open_open,close_close
year,,,
2016,sh.600031,6.45,6.10
2017,sh.600031,6.10,9.07
2018,sh.600031,9.16,8.34
2019,sh.600031,8.31,17.05
2020,sh.600031,17.29,31.50


In [59]:
c

,code_open,open_open,close_close
year,,,
2016,sh.600031,6.45,6.10
2017,sh.600031,6.10,9.07
2018,sh.600031,9.16,8.34
2019,sh.600031,8.31,17.05
2020,sh.600031,17.29,31.50
